### L0 and L1 processing of the glider data

Removing spikes and outliers

In [5]:
%pylab inline
import glidertools as gt

import sys
sys.path.append('processing_code/')

import process_seaglider_data_L1 as ps
import process_slocum_data_L1 as psl

import my_functions as my

Populating the interactive namespace from numpy and matplotlib


First load all the raw glider data files

In [1]:
%run -i 'processing_code/read_saz_glider_data.py'
%run -i 'processing_code/read_pfz_glider_data.py'
%run -i 'processing_code/read_miz_glider_data.py'


DIMENSION: ctd_data_point
{latitude, salinity_qc, ctd_pressure, longitude, salinity_raw, ctd_depth, temperature_qc, ctd_time, temperature_raw}


100%|██████████| 485/485 [00:15<00:00, 31.35it/s]


../data/gliders/SG542_SAZ_2019/p5420082.nc was skipped


Cannot merge data - not enough time indexed DataFrames
Returning unmerged dataframes

DIMENSION: ctd_data_point
{latitude, salinity_qc, ctd_pressure, longitude, salinity_raw, ctd_depth, temperature_qc, ctd_time, temperature_raw}


100%|██████████| 497/497 [00:14<00:00, 33.65it/s]


../data/gliders/SG643/p6430039.nc was skipped


Cannot merge data - not enough time indexed DataFrames
Returning unmerged dataframes


#### Subantarctic Zone

Next we do the Level 1 processing for the SAZ glider

In [ ]:
dat_saz_L1 = ps.process_seaglider_data_L1(dat_saz)

Calculate mixed layer depth and density

In [7]:
dens = my.dens(dat_saz_L1.salt, dat_saz_L1.temp, dat_saz_L1.depth, dat_saz_L1.lat, dat_saz_L1.lon, dat_saz_L1.time)

mld_01 = my.calc_mld(dens.T, dat_saz_L1.depth, den_lim=0.01)
mld_03 = my.calc_mld(dens.T, dat_saz_L1.depth, den_lim=0.03)
mld_10 = my.calc_mld(dens.T, dat_saz_L1.depth, den_lim=0.10)

MLD not calculated: profile 958. Setting to NaN
MLD not calculated: profile 959. Setting to NaN
MLD not calculated: profile 960. Setting to NaN
MLD not calculated: profile 961. Setting to NaN
MLD not calculated: profile 962. Setting to NaN
MLD not calculated: profile 963. Setting to NaN


Calculate secondary variables - using the `GliderTools` Python package, I have calculated density and MLD from the Level 1 salinity and temperature data

Assign secondary variables to a new `xarray dataset`  

In [8]:
dat_saz_L2 = dat_saz_L1.copy()

dat_saz_L2=dat_saz_L2.assign(density=(("depth", "time"), dens))
dat_saz_L2=dat_saz_L2.assign(mld_01=("time", mld_01))
dat_saz_L2=dat_saz_L2.assign(mld_03=("time", mld_03))
dat_saz_L2=dat_saz_L2.assign(mld_10=("time", mld_10))

Assigning the metadata for the SAZ Seaglider

In [9]:
from datetime import date

dat_saz_L2.attrs['instrument']              = 'Seaglider'
dat_saz_L2.attrs['owner']                   = 'Southern Ocean Carbon and Climate Observatory'
dat_saz_L2.attrs['contact']                 = 'sarahanne.n@gmail.com'
dat_saz_L2.attrs['processing_date']         = str(date.today())
dat_saz_L2.attrs['processing_level']        = '2'
dat_saz_L2.attrs['time_coverate_start']     = str(dat_saz_L2.time[0].values)
dat_saz_L2.attrs['time_coverate_end']       = str(dat_saz_L2.time[-1].values)
dat_saz_L2.attrs['geospatial_vertical_min'] = '0'
dat_saz_L2.attrs['geospatial_vertical_max'] = '1000'
dat_saz_L2.attrs['geospatial_lat_min']      = str(dat_saz_L2.lat.min().values)
dat_saz_L2.attrs['geospatial_lat_max']      = str(dat_saz_L2.lat.max().values)
dat_saz_L2.attrs['geospatial_lon_min']      = str(dat_saz_L2.lon.min().values)
dat_saz_L2.attrs['geospatial_lon_max']      = str(dat_saz_L2.lon.max().values)

#### Polar Frontal Zone Slocum

In [10]:
dat_pfz['latitude'] = (('time'), dat_pfz.latitude)
dat_pfz['longitude'] = (('time'), dat_pfz.longitude)
dat_pfz['density'] = (('time'), dat_pfz.density)

dat_pfz = dat_pfz.drop(labels=['dim_0'])

dat_pfz['ctd_depth'] = (('time'), np.abs(gsw.z_from_p(dat_pfz.ctd_pressure, dat_pfz.latitude)))

In [11]:
dat_pfz_L1 = psl.process_slocum_data_L1(dat_pfz)

Mean bin size = 5.00
Mean depth binned (50 m) vertical sampling frequency = 0.12


/Users/marcel/opt/anaconda3/lib/python3.8/site-packages/pandas/io/formats/format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
/Users/marcel/opt/anaconda3/lib/python3.8/site-packages/pandas/io/formats/format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


Mean bin size = 5.00
Mean depth binned (50 m) vertical sampling frequency = 0.12


/Users/marcel/opt/anaconda3/lib/python3.8/site-packages/pandas/io/formats/format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
/Users/marcel/opt/anaconda3/lib/python3.8/site-packages/pandas/io/formats/format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


Calculate secondary variables - using the `GliderTools` Python package, I have calculated density and MLD from the Level 1 salinity and temperature data

In [12]:
dens = my.dens(dat_pfz_L1.salt, dat_pfz_L1.temp, dat_pfz_L1.depth, dat_pfz_L1.lat, dat_pfz_L1.lon, dat_pfz_L1.time)

mld_01 = my.calc_mld(dens.T, dat_pfz_L1.depth, den_lim=0.01)
mld_03 = my.calc_mld(dens.T, dat_pfz_L1.depth, den_lim=0.03)
mld_10 = my.calc_mld(dens.T, dat_pfz_L1.depth, den_lim=0.10)

Assign secondary variables to a new `xarray dataset`  

In [13]:
dat_pfz_L2 = dat_pfz_L1.copy()

dat_pfz_L2 = dat_pfz_L2.assign(density=(("depth", "time"), dens))
dat_pfz_L2 = dat_pfz_L2.assign(mld_01=("time", mld_01))
dat_pfz_L2 = dat_pfz_L2.assign(mld_03=("time", mld_03))
dat_pfz_L2 = dat_pfz_L2.assign(mld_10=("time", mld_10))

In [14]:
from datetime import date

dat_pfz_L2.attrs['instrument']              = 'Webb Teledyne Slocum'
dat_pfz_L2.attrs['owner']                   = 'Southern Ocean Carbon and Climate Observatory'
dat_pfz_L2.attrs['contact']                 = 'sarahanne.n@gmail.com'
dat_pfz_L2.attrs['processing_date']         = str(date.today())
dat_pfz_L2.attrs['processing_level']        = '2'
dat_pfz_L2.attrs['time_coverate_start']     = str(dat_pfz_L2.time[0].values)
dat_pfz_L2.attrs['time_coverate_end']       = str(dat_pfz_L2.time[-1].values)
dat_pfz_L2.attrs['geospatial_vertical_min'] = '0'
dat_pfz_L2.attrs['geospatial_vertical_max'] = '1000'
dat_pfz_L2.attrs['geospatial_lat_min']      = str(dat_pfz_L2.lat.min().values)
dat_pfz_L2.attrs['geospatial_lat_max']      = str(dat_pfz_L2.lat.max().values)
dat_pfz_L2.attrs['geospatial_lon_min']      = str(dat_pfz_L2.lon.min().values)
dat_pfz_L2.attrs['geospatial_lon_max']      = str(dat_pfz_L2.lon.max().values)

### Marginal Ice Zone Seaglider

Run the Level 1 processing script

In [15]:
dat_miz_L1 = ps.process_seaglider_data_L1(dat_miz)

Mean bin size = 5.00
Mean depth binned (50 m) vertical sampling frequency = 0.56


/Users/marcel/opt/anaconda3/lib/python3.8/site-packages/pandas/io/formats/format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
/Users/marcel/opt/anaconda3/lib/python3.8/site-packages/pandas/io/formats/format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


Mean bin size = 5.00
Mean depth binned (50 m) vertical sampling frequency = 0.56


/Users/marcel/opt/anaconda3/lib/python3.8/site-packages/pandas/io/formats/format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())
/Users/marcel/opt/anaconda3/lib/python3.8/site-packages/pandas/io/formats/format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


Calculate secondary variables - using the `GliderTools` Python package, I have calculated density and MLD from the Level 1 salinity and temperature data

In [16]:
dens = my.dens(dat_miz_L1.salt, dat_miz_L1.temp, dat_miz_L1.depth, dat_miz_L1.lat, dat_miz_L1.lon, dat_miz_L1.time)

mld_01 = my.calc_mld(dens.T, dat_miz_L1.depth, den_lim=0.01)
mld_03 = my.calc_mld(dens.T, dat_miz_L1.depth, den_lim=0.03)
mld_10 = my.calc_mld(dens.T, dat_miz_L1.depth, den_lim=0.10)

MLD not calculated: profile 347. Setting to NaN
MLD not calculated: profile 484. Setting to NaN
MLD not calculated: profile 347. Setting to NaN
MLD not calculated: profile 484. Setting to NaN
MLD not calculated: profile 347. Setting to NaN
MLD not calculated: profile 484. Setting to NaN


Assign secondary variables to a new `xarray dataset`  

In [17]:
dat_miz_L2 = dat_miz_L1.copy()

dat_miz_L2 = dat_miz_L2.assign(density=(("depth", "time"), dens))
dat_miz_L2 = dat_miz_L2.assign(mld_01=("time", mld_01))
dat_miz_L2 = dat_miz_L2.assign(mld_03=("time", mld_03))
dat_miz_L2 = dat_miz_L2.assign(mld_10=("time", mld_10))

In [18]:
from datetime import date

dat_miz_L2.attrs['instrument']              = 'Hydroid Seaglider'
dat_miz_L2.attrs['owner']                   = 'University of Gothenburg'
dat_miz_L2.attrs['contact']                 = 'sebastiaan.swart@marine.gu.se'
dat_miz_L2.attrs['processing_date']         = str(date.today())
dat_miz_L2.attrs['processing_level']        = '2'
dat_miz_L2.attrs['time_coverate_start']     = str(dat_miz_L2.time[0].values)
dat_miz_L2.attrs['time_coverate_end']       = str(dat_miz_L2.time[-1].values)
dat_miz_L2.attrs['geospatial_vertical_min'] = '0'
dat_miz_L2.attrs['geospatial_vertical_max'] = '1000'
dat_miz_L2.attrs['geospatial_lat_min']      = str(dat_miz_L2.lat.min().values)
dat_miz_L2.attrs['geospatial_lat_max']      = str(dat_miz_L2.lat.max().values)
dat_miz_L2.attrs['geospatial_lon_min']      = str(dat_miz_L2.lon.min().values)
dat_miz_L2.attrs['geospatial_lon_max']      = str(dat_miz_L2.lon.max().values)

Save all data as NetCDF to the data folder

In [19]:
dat_saz.to_netcdf   ('../data/sg542_saz_L0.nc')
dat_saz_L1.to_netcdf('../data/sg542_saz_L1.nc')
dat_saz_L2.to_netcdf('../data/sg542_saz_L2.nc')

dat_pfz.to_netcdf   ('../data/slocum_pfz_L0.nc')
dat_pfz_L1.to_netcdf('../data/slocum_pfz_L1.nc')
dat_pfz_L2.to_netcdf('../data/slocum_pfz_L2.nc')

dat_miz.to_netcdf   ('../data/sg643_miz_L0.nc')
dat_miz_L1.to_netcdf('../data/sg643_miz_L1.nc')
dat_miz_L2.to_netcdf('../data/sg643_miz_L2.nc')